# Intelligent Agents: Reflex Agents for the Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'north'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: east
step 1 - action: suck
step 2 - action: suck
step 3 - action: suck
step 4 - action: east
step 5 - action: south
step 6 - action: north
step 7 - action: east
step 8 - action: north
step 9 - action: west
step 10 - action: east
step 11 - action: suck
step 12 - action: west
step 13 - action: north
step 14 - action: east
step 15 - action: west
step 16 - action: north
step 17 - action: east
step 18 - action: east
step 19 - action: west


4

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [5]:
def simulation_environment(n, agent):
    # Create n * n square room
    # Dirt is randomly placed on each square with probability p = 0.2
    # 0 means clean and 1 means dirty
    square_state = [[np.random.choice([0, 1], p = [0.8, 0.2]) for i in range(n)] for j in range(n)]
    
    # Count the number of dirt
    num_dirty = 0
    for i in range(0, n):
        for j in range(0, n):
            if square_state[i][j] == 1:
                num_dirty += 1
    
    # Put the agent on a random location
    x_position = np.random.randint(0, n)
    y_position = np.random.randint(0, n)
    
    # Check if the agent is next to the wall
    bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
    if x_position == 0: bumpers["north"] = True
    elif x_position == n - 1: bumpers["south"] = True
    
    if y_position == 0: bumpers["west"] = True
    elif y_position == n - 1: bumpers["east"] = True
        
    # Count total step
    step = 0
    
    while num_dirty != 0:
        dirty = square_state[x_position][y_position]
        action = agent(bumpers, dirty)
        if action == "suck" and dirty == 1:
            # Remove the dirty if agent do the action 'suck' on a dirty square
            square_state[x_position][y_position] = 0
            num_dirty -= 1
        elif action == "north":
            if bumpers["north"] == False: x_position -= 1
            else: continue
        elif action == "south":
            if bumpers["south"] == False: x_position += 1 
            else: continue
        elif action == "west":
            if bumpers["west"] == False: y_position -= 1
            else: continue
        elif action == "east":
            if bumpers["east"] == False: y_position += 1
            else: continue
            
        # Check if the agent have reached the wall
        if x_position == 0: bumpers["north"] = True
        elif x_position == n - 1: bumpers["south"] = True
        else:
            bumpers["north"] = False
            bumpers["south"] = False
            
        if y_position == 0: bumpers["west"] = True
        elif y_position == n - 1: bumpers["east"] = True
        else:
            bumpers["west"] = False
            bumpers["east"] = False
                
        step += 1
        
        #if agent can't finish cleaning in 100,000 steps, return -1 means not complete cleaning
        if step > 100000:
            step = -1
            break
            
    return step

In [6]:
simulation_environment(5, simple_randomized_agent)

242

## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ The agent cannot directly use variable in the environment. It only gets the percepts as the arguments to the agent program function.

In [6]:
def simple_reflex_agent(bumpers, dirty):
    
    actions = []
    
    if dirty:
        action = "suck"
    else:
        if bumpers["north"] == False: actions.append("north")
        if bumpers["south"] == False: actions.append("south")
        if bumpers["west"] == False: actions.append("west")
        if bumpers["east"] == False: actions.append("east")
        action = np.random.choice(actions)
        
    return action

## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determine its location and then is able to use more advanced navigation.

Describe how you define the __agent state__ and how your agent works before implementing it. _Note on implementing the state in Python:_ [Examples](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb)

## Answer
Agent goes to the SE corner at the begining, and uses global variable 'state_arrive_corner' to record if it have arrived SE corner. Then agent goes west to the wall and then north to the wall to measure the length and width of the room.

Agent uses global variable 'state_scan_complete' to mark that it has completed measuring the room. Agent uses global variable 'state_room_length' and  state_room_width' to record the length and width of the room. Agent uses global variable 'state_x_position' and 'state_y_position' to record its own position. Agent uses global variable 'square_state_memory' to remember where it has arrived and cleaned. Agent cleans the room in a S-shaped way from north to south.

In [8]:
state_arrive_corner = False
state_scan_complete = False
state_room_length = 1
state_room_width = 1
state_x_position = -1
state_y_position = -1
square_state_memory = [[0 for i in range(state_room_length)] for j in range(state_room_width)]

def model_based_agent(bumpers, dirty):
    
    global state_room_length
    global state_room_width
    global state_arrive_corner
    global state_scan_complete
    global state_x_position
    global state_y_position
    global square_state_memory
    
    # Move to the SE corner first to determine its location
    while state_arrive_corner == False and bumpers["south"] != True:
        action = "south"
        return action 
    
    while state_arrive_corner == False and bumpers["south"] == True and bumpers["east"] != True:
        action = "east"
        return action
    
    if bumpers["south"] == True and bumpers["east"] == True:
        state_arrive_corner = True
        
    # Scan the length and width of the room by moveing to NW corner from SE corner
    while state_arrive_corner == True and state_scan_complete == False and bumpers["west"] != True:
        action = "west"
        state_room_length += 1
        return action
    
    while state_arrive_corner == True and state_scan_complete == False and bumpers["north"] != True:
        action = "north"
        state_room_width += 1
        return action
    
    if state_arrive_corner == True and state_scan_complete == False and bumpers["north"] == True and bumpers["west"] == True:
        action = "scan completed"
        state_scan_complete = True
        
        # Init agent's memeory of square state
        # 0 means not reach, 1 means reached but no need to clean, 2 means reached and cleaned
        square_state_memory = [[0 for i in range(state_room_length)] for j in range(state_room_width)]
        
        # Init agent's memory of its position
        state_x_position = 0
        state_y_position = 0
        
        return action
    
    # Agent cleans the room row by row
    if state_arrive_corner == True and state_scan_complete == True:
        
        if square_state_memory[state_x_position][state_y_position] != 2:
            square_state_memory[state_x_position][state_y_position] = 1
            
        if dirty == 1:
            square_state_memory[state_x_position][state_y_position] = 2
            action = "suck"
            
        elif state_x_position % 2 == 0 and state_y_position != state_room_length - 1:
            action = "east"
            state_y_position += 1
            
        elif state_x_position % 2 == 1 and state_y_position != 0:
            action = "west"
            state_y_position -= 1
            
        elif state_x_position % 2 == 0 and state_y_position == state_room_length - 1:
            state_x_position += 1
            action = "south"
            
        elif state_x_position % 2 == 1 and state_y_position == 0:
            state_x_position += 1
            action = "south"   
            
        return action

## Task 4: Simulation study [3 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences. 

Here is some help with [charts and tables.](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb)

In [9]:
import pandas as pd

# Simulate in a 5 * 5 room
sum_step_simple_randomized_agent = 0
sum_step_simple_reflex_agent = 0
sum_step_model_based_agent = 0

step_simple_randomized_agent = [0 for i in range(100)]
step_simple_reflex_agent = [0 for i in range(100)]
step_model_based_agent = [0 for i in range(100)]

for i in range(100):
    # Clear model based agent's memory
    state_arrive_corner = False
    state_scan_complete = False
    state_room_length = 1
    state_room_width = 1
    state_x_position = -1
    state_y_position = -1

    step_simple_randomized_agent[i] = simulation_environment(5, simple_randomized_agent)
    step_simple_reflex_agent[i] = simulation_environment(5, simple_reflex_agent)
    step_model_based_agent[i] = simulation_environment(5, model_based_agent)

    sum_step_simple_randomized_agent += step_simple_randomized_agent[i]
    sum_step_simple_reflex_agent += step_simple_reflex_agent[i]
    sum_step_model_based_agent += step_model_based_agent[i]

print("Simple randomized agent average performance in %d * %d: " % (5, 5), int(sum_step_simple_randomized_agent/100), "steps")
print("Simple reflex agent average performance in %d * %d: " % (5, 5), int(sum_step_simple_reflex_agent/100), "steps")
print("Model based agent average performance in %d * %d: " % (5, 5), int(sum_step_model_based_agent/100), "steps")
print("")

performance = {
    "simple randomized agent" : step_simple_randomized_agent,
    "simple reflex agent" : step_simple_reflex_agent,
    "model based agent" : step_model_based_agent 
}

df = pd.DataFrame(performance)
df

Simple randomized agent average performance in 5 * 5:  369 steps
Simple reflex agent average performance in 5 * 5:  96 steps
Model based agent average performance in 5 * 5:  37 steps



,simple randomized agent,simple reflex agent,model based agent
0,220,67,38
1,355,159,0
2,265,32,34
3,203,40,39
4,76,224,41
...,...,...,...
95,485,48,40
96,143,113,23
97,318,135,40
98,186,85,39


In [10]:
# Simulate in a 10 * 10 room
sum_step_simple_randomized_agent = 0
sum_step_simple_reflex_agent = 0
sum_step_model_based_agent = 0

step_simple_randomized_agent = [0 for i in range(100)]
step_simple_reflex_agent = [0 for i in range(100)]
step_model_based_agent = [0 for i in range(100)]

for i in range(100):
    # Clear model based agent's memory
    state_arrive_corner = False
    state_scan_complete = False
    state_room_length = 1
    state_room_width = 1
    state_x_position = -1
    state_y_position = -1
    
    step_simple_randomized_agent[i] = simulation_environment(10, simple_randomized_agent)
    step_simple_reflex_agent[i] = simulation_environment(10, simple_reflex_agent)
    step_model_based_agent[i] = simulation_environment(10, model_based_agent)
    
    sum_step_simple_randomized_agent += step_simple_randomized_agent[i]
    sum_step_simple_reflex_agent += step_simple_reflex_agent[i]
    sum_step_model_based_agent += step_model_based_agent[i]
    
print("Simple randomized agent average performance in 10 * 10: ", int(sum_step_simple_randomized_agent/100), "steps")
print("Simple reflex agent average performance in 10 * 10: ", int(sum_step_simple_reflex_agent/100), "steps")
print("Model based agent average performance in 10 * 10: ", int(sum_step_model_based_agent/100), "steps")
print("")

performance = {
    "simple randomized agent" : step_simple_randomized_agent,
    "simple reflex agent" : step_simple_reflex_agent,
    "model based agent" : step_model_based_agent 
}

df = pd.DataFrame(performance)
df

Simple randomized agent average performance in 10 * 10:  2512 steps
Simple reflex agent average performance in 10 * 10:  910 steps
Model based agent average performance in 10 * 10:  142 steps



,simple randomized agent,simple reflex agent,model based agent
0,2369,1560,119
1,4146,1596,147
2,2043,380,157
3,3104,503,155
4,3024,1337,147
...,...,...,...
95,2830,698,143
96,2107,620,127
97,1900,466,146
98,1952,738,138


In [11]:
# Simulate in a 100 * 100 room
sum_step_simple_randomized_agent = 0
sum_step_simple_reflex_agent = 0
sum_step_model_based_agent = 0

step_simple_randomized_agent = [0 for i in range(100)]
step_simple_reflex_agent = [0 for i in range(100)]
step_model_based_agent = [0 for i in range(100)]

for i in range(100):
    # Clear model based agent's memory
    state_arrive_corner = False
    state_scan_complete = False
    state_room_length = 1
    state_room_width = 1
    state_x_position = -1
    state_y_position = -1
    
    step_simple_randomized_agent[i] = simulation_environment(100, simple_randomized_agent)
    step_simple_reflex_agent[i] = simulation_environment(100, simple_reflex_agent)
    step_model_based_agent[i] = simulation_environment(100, model_based_agent)
    
    sum_step_simple_randomized_agent += step_simple_randomized_agent[i]
    sum_step_simple_reflex_agent += step_simple_reflex_agent[i]
    sum_step_model_based_agent += step_model_based_agent[i]
    
print("Simple randomized agent average performance in 100 * 100: ", int(sum_step_simple_randomized_agent/100), "steps")
print("Simple reflex agent average performance in 100 * 100: ", int(sum_step_simple_reflex_agent/100), "steps")
print("Model based agent average performance in 100 * 100: ", int(sum_step_model_based_agent/100), "steps")
print("")

performance = {
    "simple randomized agent" : step_simple_randomized_agent,
    "simple reflex agent" : step_simple_reflex_agent,
    "model based agent" : step_model_based_agent 
}

df = pd.DataFrame(performance)
df

Simple randomized agent average performance in 100 * 100:  -1 steps
Simple reflex agent average performance in 100 * 100:  -1 steps
Model based agent average performance in 100 * 100:  12296 steps



,simple randomized agent,simple reflex agent,model based agent
0,-1,-1,12397
1,-1,-1,12337
2,-1,-1,12282
3,-1,-1,12389
4,-1,-1,12333
...,...,...,...
95,-1,-1,12357
96,-1,-1,12343
97,-1,-1,12302
98,-1,-1,12290


Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | 369 | 96 | 37 |
| 10x10   | 2512 | 910 | 142 |
| 100x100 | > 100,000 | > 100,000 | 12296 |

# Answer
We can see from the table and the chart that simple randomized agent always uses three times as many steps as simple reflex agent does. And steps model besed agent using grows much slower than that of simple randomized agent and simple reflex agent. Steps model based agent using grows linear with growing squares of the room.

We can see from the table and the chart that simple randomized agent always uses three times as many steps as simple reflex agent does. And steps model besed agent using grows much slower than that of simple randomized agent and simple reflex agent. Steps model based agent using grows linear with squares that the room has.

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

# Answer
If put the agent into a rectangular room with unknown size: Simple randomized agent will complete the cleaning successfully. Simple reflex agent will complete the cleaning successfully. Model based agent will complete the cleaning successfully.

If the cleaning area has an iregular shape: Simple randomized agent will complete the cleaning successfully. Simple reflex agent will complete the cleaning successfully. Model based agent will cleaning only part of the area.

If the room contains obstacles: Simple randomized agent will complete the cleaning successfully. Simple reflex agent will complete the cleaning successfully. Model based agent will not complete the cleaning successfully.

## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [15]:
def obstacles_environment(n, agent):
    # Create n * n square room
    # Dirt is randomly placed on each square with probability p = 0.2
    # Obstacle is randomly placed on each square with probability p = 0.1
    # 0 means clean, 1 means dirty, 2 means obstacle
    square_state = [[np.random.choice([0, 1, 2], p = [0.7, 0.2, 0.1]) for i in range(n)] for j in range(n)]
    
    # Count the number of dirt
    num_dirty = 0
    for i in range(0, n):
        for j in range(0, n):
            if square_state[i][j] == 1:
                num_dirty += 1
    
    # Put the agent on a random location
    x_position = np.random.randint(0, n)
    y_position = np.random.randint(0, n)
    
    # Check if the agent is next to the wall or an obstacle
    bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
    if (x_position == 0): 
        bumpers["north"] = True
        if (square_state[x_position + 1][y_position] == 2):
            bumpers["south"] = True
    elif (x_position == n - 1): 
        bumpers["south"] = True
        if (square_state[x_position - 1][y_position] == 2):
            bumpers["north"] = True
    else:
        if (square_state[x_position + 1][y_position] == 2):
            bumpers["south"] = True
        if (square_state[x_position - 1][y_position] == 2):
            bumpers["north"] = True
    
    if (y_position == 0): 
        bumpers["west"] = True
        if (square_state[x_position][y_position + 1] == 2):
            bumpers["east"] = True
    elif (y_position == n - 1): 
        bumpers["east"] = True
        if (square_state[x_position][y_position - 1] == 2):
            bumpers["west"] = True
    else:
        if (square_state[x_position][y_position + 1] == 2):
            bumpers["east"] = True
        if (square_state[x_position][y_position - 1] == 2):
            bumpers["west"] = True
        
    # Count total step
    step = 0
    
    while num_dirty != 0:
        dirty = square_state[x_position][y_position]
        action = agent(bumpers, dirty)
        if action == "suck" and dirty == 1:
            # Remove the dirty if agent do the action 'suck' on a dirty square
            square_state[x_position][y_position] = 0
            num_dirty -= 1
        elif action == "north":
            if bumpers["north"] == False: x_position -= 1
            else: continue
        elif action == "south":
            if bumpers["south"] == False: x_position += 1 
            else: continue
        elif action == "west":
            if bumpers["west"] == False: y_position -= 1
            else: continue
        elif action == "east":
            if bumpers["east"] == False: y_position += 1
            else: continue
            
        
        # Check if the agent is next to the wall or an obstacle
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
        if (x_position == 0): 
            bumpers["north"] = True
            if (square_state[x_position + 1][y_position] == 2):
                bumpers["south"] = True
        elif (x_position == n - 1): 
            bumpers["south"] = True
            if (square_state[x_position - 1][y_position] == 2):
                bumpers["north"] = True
        else:
            if (square_state[x_position + 1][y_position] == 2):
                bumpers["south"] = True
            if (square_state[x_position - 1][y_position] == 2):
                bumpers["north"] = True

        if (y_position == 0): 
            bumpers["west"] = True
            if (square_state[x_position][y_position + 1] == 2):
                bumpers["east"] = True
        elif (y_position == n - 1): 
            bumpers["east"] = True
            if (square_state[x_position][y_position - 1] == 2):
                bumpers["west"] = True
        else:
            if (square_state[x_position][y_position + 1] == 2):
                bumpers["east"] = True
            if (square_state[x_position][y_position - 1] == 2):
                bumpers["west"] = True
                
        step += 1
        
        #if agent can't finish cleaning in 100,000 steps, return -1 means not complete cleaning
        if (step > 100000):
            step = -1
            break
            
    return step

In [16]:
print(obstacles_environment(5, simple_randomized_agent))
print(obstacles_environment(5, simple_reflex_agent))

37
113


# Answer
If we add random obstacle squares in the room: Simple randomized agent will complete cleaning successfully. Simple reflex agent will complete cleaning successfully. Model based agent will fail to respond to obstacles successfully and cannot finish cleaning

If we want our model based agent perform better with obstacles: We should add obstacles recognize function in it. Which mean after the agent finish scanning the room can start to clean, if it senses an unexpected bumper in its way, it should bypass the obstacle by moving like south-east-east-north if there is an obstacles on its east. And then it will go back to it expected route.

## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities  Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!

# Unknown environment with obstacles
Record each square's state in the agent, before each step, scan the squares around the agent which are one step away from the agent. If there is an unchecked or uncleaned square, agent goes to it. If there isn't an one step square, then search all the two step away square. If yes, go to it. If no, then search three step squares...

# Utility-based agent
Using the function that dealing with unknown environment with obstacles above to always go to the nearest dirty sqaure.